# MBA em Data Science & Analytics
##USP/ESALQ
###Raphael Mandich



Este notebook tem como objetivo conter os resultados preliminares do trabalho de conclusão de curso do MBA em Data Science & Analytics. O propósito do estudo é construir um algoritmo capaz de resolver uma instância de um problema de roteamento de veículos capacitado, em inglês Capacitated Vehicle Routing Problem (CVRP), utilizando dados e funções disponíveis no repositório LoggiBUD. O CVRP pode ser encarado como uma generalização de um problema mais famoso na literatura de análise combinatória e pesquisa operacional, o Problema do Caixeiro Viajante, em inglês Traveling Salesman Problem(TSP).



In [1]:
# Clonagem do repositório
!git clone https://github.com/loggi/loggibud &> /dev/null
# Comando "&> /dev/null" oculta o output

In [3]:
# Acessando a pasta do repositório
%cd /content/loggibud/

/content/loggibud


In [4]:
# Instalação das dependências do repositório
!pip install poetry &> /dev/null
!poetry install &> /dev/null
!poetry export -f requirements.txt --without-hashes --output requirements.txt &> /dev/null
!pip install -r requirements.txt &> /dev/null

In [ ]:
# Testes de verificação do repositório
!poetry run pytest -s -v tests/

============================= test session starts ==============================
platform linux -- Python 3.7.13, pytest-6.2.5, py-1.10.0, pluggy-1.0.0 -- /root/.cache/pypoetry/virtualenvs/loggibud-3SbdMl6d-py3.7/bin/python
cachedir: .pytest_cache
rootdir: /content/loggibud
collected 7 items / 1 skipped / 6 selected                                     

tests/v1/test_data_conversion.py::test_can_create_proper_tsplib_from_instance PASSED
tests/v1/test_distances.py::test_great_circle_distance PASSED
tests/v1/test_distances.py::test_great_circle_route_distance PASSED
tests/v1/test_task1_baselines.py::test_ortools_solver PASSED
tests/v1/test_task1_baselines.py::test_lkh_solver PASSED
100% 158/158 [00:00<00:00, 2225.01it/s]
PASSED
100% 158/158 [00:00<00:00, 60245.46it/s]
PASSED

========================= 7 passed, 1 skipped in 9.14s =========================


O repositório foi instalado corretamente, podemos seguir com a carga dos dados

In [6]:
# Baixa o arquivo zip com os dados compilados
!wget -nc https://loggibud.s3.amazonaws.com/dataset.zip &> /dev/null
# Descompactação do arquivo
!unzip -n dataset.zip &> /dev/null

Os dados de entregas contem arquivos no formato .json divididos em dia e região onde as entregas ocorrerão. Cada instância é uma variável do tipo CVRPInstance, e contém as propriedades name(Nome da Instância), region (Região da Instância), origin(Latitude e Longitude do ponto de origem), vehicle_capacity(Capacidade do veículo de entrega) e deliveries(Lista com todas as entregas da instância). Podemos observar melhor a estrutura de dados importando a biblioteca e carregando uma instância de exemplo, conforme abaixo:

In [ ]:
# Importa a biblioteca CVRPInstance do repositório
from loggibud.v1.types import CVRPInstance

# Carrega uma instância em um objeto 
path = "./data/cvrp-instances-1.0/train/df-0/cvrp-0-df-0.json"
instancia = CVRPInstance.from_file(path)

Podemos visualizar todas as entregas de uma instância no mapa utilizando a função plot_cvrp_instance

In [ ]:
# Importa a função do repositório
from loggibud.v1.plotting.plot_instance import plot_cvrp_instance


plot_cvrp_instance(instancia)

Cada ponto do mapa acima é um pedido com um determinado "tamanho", e todas as rotas partem do mesmo ponto de partida. Para entregá-las, serão utilizados diversos veículos, cada um com uma capacidade para suportar um número de entregas.

O objetivo é organizar cada pedido em um veículo e construir rotas para entregá-los da forma mais eficiente possível. O repositório possui alguns algoritmos prontos para resolver esse tipo de problema, como por exemplo o LKH-3, utilizado abaixo:

In [ ]:
# Importa o solver lkh-3
from loggibud.v1.baselines.task1 import lkh_3
# Importa o módulo de cálculo de distâncias
from loggibud.v1.distances import OSRMConfig

# Utilização do OSRM para calcular as distâncias entre um pedido e outro
osrm_config = OSRMConfig(host="http://ec2-34-222-175-250.us-west-2.compute.amazonaws.com")
lkh_params = lkh_3.LKHParams(osrm_config=osrm_config)

# Armazena o resultado do solver em um objeto
solution = lkh_3.solve(instancia, params=lkh_params)
print(f"A solução encontrada precisou de {len(solution.vehicles)} veículos/rotas")

A solução encontrada precisou de 32 veículos/rotas


Para organizar os pedidos em rotas, é necessário saber as distâncias entre um e o outro. Como estamos lidando com pontos urbanos, não é possível utilizar medidas de distância mais teóricas, como a distância Euclidiana por exemplo, que dá a distância em linha reta entre dois pontos. A distância Euclidiana não atende nossos propósitos, já que estamos em uma cidade, e precisamos nos movimentar por ruas, e não em linha reta.

Por isso, utilizamos um serviço chamado Open Source Routing Machine (OSRM), que retorna a distância de rua em vez da Euclidiana. No exemplo acima, a variável osrm_config armazena seu endereço e o solver calcula a melhor rota baseado nessas distâncias.

Podemos visualizar melhor a solução obtida através da plotagem no mapa

In [ ]:
from loggibud.v1.plotting.plot_solution import plot_cvrp_solution

# Plota as rotas da solução em linha reta
plot_cvrp_solution(solution)

Ou podemos utilizar uma função ligeiramente diferente para visualizar as mesmas rotas traçadas nas ruas

In [ ]:
from loggibud.v1.plotting.plot_solution import plot_cvrp_solution_routes

# Plota as rotas nas ruas do mapa
plot_cvrp_solution_routes(solution, config=osrm_config)

Além de visualizar as rotas graficamente, podemos avaliar a distância total percorrida em Kilometros. No caso a distância total é de pouco mais de 1497km.

In [ ]:
from loggibud.v1.eval.task1 import evaluate_solution


evaluate_solution(instancia, solution, config=osrm_config)

1497.7241

O exemplo avaliado assume que todas as entregas já estão disponíveis no ponto de partida. Porém, na realidade de empresas de transporte, normalmente não há espaço suficiente para comportar todas as entregas de um período. Além disso, os pedidos podem chegar em momentos diferentes, e não é eficiente deixar entregas que já chegaram esperando para então executar o algoritmo. 

Devido a essas e outras restrições operacionais, é necessário que o algoritmo seja capaz de atribuir uma entrega a um veículo à medida em que ela torna-se disponível. O problema anterior é conhecido na literatura como "TSP Estático", já que toda informação necessária está disponível desde o início e mantém-se fixa. Já o problema em que a informação fica disponível com o tempo é denominado "TSP Dinâmico".

Apesar de ser um problema relativamente simples, o TSP é um problema de difícil resolução. Conforme o número de nós cresce, o tempo gasto para resolver o problema cresce exponencialmente. Por isso, é mais viável utilizar métodos que retornam uma boa rota em vez da melhor possível, mas que não demandem tempos impraticáveis. Para melhor visualizar essa interação, podemos tentar utilizar um algoritmo simples de força bruta para resolver um TSP genérico.


Iniciaremos criando uma simples matriz contendo alguns números que servirão como medidas de distância

In [ ]:
import numpy as np

# Cria uma matriz 4x4 que servirá como exemplo de distâncias
matriz_distancias = np.array([
    [0, 7, 20, 6],
    [7, 0, 12, 14],
    [20, 12, 0, 8],
    [6, 14, 8, 0],
])
matriz_distancias

array([[ 0,  7, 20,  6],
       [ 7,  0, 12, 14],
       [20, 12,  0,  8],
       [ 6, 14,  8,  0]])

Esta matriz significa que a distância do nó `i` ao nó `j` é dada por `matriz_distancia[i, j]`. Por exemplo, a distância entre [0,1] é 7, a distância entre [1,2] é 20 e assim sucessivamente.

A partir dessa matriz, podemos criar rotas de alguns pontos e utilizá-la como base no cálculo de distância, conforme abaixo

In [ ]:
# 0 -> 1 -> 3 -> 2 -> 0
rota1 = [0, 1, 3, 2, 0]
rota1

[0, 1, 3, 2, 0]

In [ ]:
# Soma a distância entre os pontos da matriz percorridos pela rota 
distancia_rota1 = (
    matriz_distancias[rota1[0], rota1[1]]
    + matriz_distancias[rota1[1], rota1[2]]
    + matriz_distancias[rota1[2], rota1[3]]
    + matriz_distancias[rota1[3], rota1[4]]
)
distancia_rota1

49

Podemos deixar o código mais genérico para que funcione com qualquer número de nós, iterando entre as distâncias do nó `i` ao nó `i + 1`

In [ ]:
def computa_distancia(matriz_distancias, rota):
    """
    Calcula a distância total de uma rota para uma dada matriz de distâncias
    """
    distancia_rota = 0
    for i in range(len(rota1) - 1):
      distancia_rota += matriz_distancias[rota[i], rota[i + 1]]

    return distancia_rota

Aplicando a função para a rota definida anteriormente temos o mesmo resultado

In [ ]:
distancia_rota1 = computa_distancia(matriz_distancias, rota1)
print(f"A distância total da rota {rota1} é de {distancia_rota1} km")

A distância total da rota [0, 1, 3, 2, 0] é de 49 km


Com a função definida podemos construir um algoritmo que, dada uma matriz de distância n X n, testa todas as possíveis rotas e retorna a mais curta. Cada rota será composta pelo nó 0, seguido de uma permutação de elementos de 1 a n, seguida novamente do nó 0. O algoritmo testa cada permutação possível e determina aquela com a rota mais curta.

In [ ]:
# Biblioteca que testa permutações possíveis
from itertools import permutations


def tsp_brute_force(matriz_distancias):
    """Resolve o TSP por meio de um algoritmo de força bruta"""
    n = matriz_distancias.shape[0]  # número de nós do problema
    melhor_rota = None  # inicializa melhor rota como elemento vazio
    distancia_melhor_rota = np.inf  # distância inicial é um número muito alto
    # todas as permutações de 1 a n-1
    for permutation in permutations(range(1, n)):
        rota = [0] + list(permutation) + [0]
        distancia_rota = computa_distancia(matriz_distancias, rota)

        # Se a permutação possuir uma distância total menor que a atual, 
        # substituir por ela
        if distancia_rota < distancia_melhor_rota:
            melhor_rota = rota
            distancia_melhor_rota = distancia_rota
  
    return melhor_rota, distancia_melhor_rota

def computa_distancia(matriz_distancias, rota):
    """
    Calcula a distância total de uma rota para uma dada matriz de distâncias
    """
    distancia_rota = 0
    for i in range(len(rota1) - 1):
      distancia_rota += matriz_distancias[rota[i], rota[i + 1]]

    return distancia_rota


# Testando com a matriz de distância definida anteriormente
tsp_brute_force(matriz_distancias)

([0, 1, 2, 3, 0], 33)

Embora o algoritmo acima funcione perfeitamente, ele não é capaz de resolver os problemas em tempo factível conforme aumentamos o número de nós da matriz de distâncias. Para problemas com 12 nós o algoritmo já leva alguns minutos para retornar o resultado, e o tempo de processamento aumenta muito a cada nó adicional. 

Vamos então utilizar o solver comercial OR-Tools, que apresenta soluções heurísticas para o TSP.

In [ ]:
!pip install ortools==9.0.9048

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Inicialmente precisamos implementar um gerenciador que converterá cada um dos nós da matriz de distâncias em índices internos do código.

In [ ]:
from ortools.constraint_solver import pywrapcp

# Número de nós do problema
n = matriz_distancias.shape[0] 
# Número de veículos 
num_veiculos = 1 
# Índice do nó que representa o ponto de origem 
no_inicial = 0 

gerenciador = pywrapcp.RoutingIndexManager(n, num_veiculos, no_inicial)
gerenciador

<ortools.constraint_solver.pywrapcp.RoutingIndexManager; proxy of <Swig Object of type 'operations_research::RoutingIndexManager *' at 0x7fcba3ba4de0> >

O segundo passo é implementar uma variável que armazenará o modelo de roteamento utilizado

In [ ]:
roteamento = pywrapcp.RoutingModel(gerenciador)
roteamento

<ortools.constraint_solver.pywrapcp.RoutingModel; proxy of <Swig Object of type 'operations_research::RoutingModel *' at 0x7fcba3ba4e10> >

O terceiro e último passo é definir uma função que recebe dois índices e retorna a distância entre eles de acordo com a matriz de distâncias

In [ ]:
def retorna_distancia(i, j):
    ni = gerenciador.IndexToNode(i)
    nj = gerenciador.IndexToNode(j)
    return matriz_distancias[ni, nj]

transit_callback_index = roteamento.RegisterTransitCallback(retorna_distancia)
roteamento.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

Com isso estamos prontos para resolver o problema utilizando o OR-Tools

In [ ]:
parametros_busca = pywrapcp.DefaultRoutingSearchParameters()
solucao_ort = roteamento.SolveWithParameters(parametros_busca)
solucao_ort.ObjectiveValue()

33

Nesse caso, o OR-Tools retornou a mesma resposta do algoritmo de força bruta, ou seja, a solução ótima. Porém, enquanto o algoritmo de força bruta tem garantia de retornar a melhor solução possível, o OR-Tools usa métodos heurísticos e portanto não existe esta certeza. Para problemas relativamente pequenos, no entanto, as soluções apresentadas tendem a ser otimizadas.

O código abaixo encampsula todo o processo anterior em uma função que resolve o TSP utilizando o OR-Tools, e retorna a rota construída e a distância total percorrida.

In [ ]:
def tsp_ortools(matriz_distancias):
    # Número de nós do problema
    n = matriz_distancias.shape[0] 
    # Número de veículos 
    num_veiculos = 1 
    # Índice do nó que representa o ponto de origem 
    no_inicial = 0 
    gerenciador = pywrapcp.RoutingIndexManager(n, num_veiculos, no_inicial)
    roteamento = pywrapcp.RoutingModel(gerenciador)
    
    def retorna_distancia(i, j):
        ni = gerenciador.IndexToNode(i)
        nj = gerenciador.IndexToNode(j)
        return matriz_distancias[ni, nj]

    transit_callback_index = roteamento.RegisterTransitCallback(retorna_distancia)
    roteamento.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Resolve o problema com métodos padrão
    parametros_busca = pywrapcp.DefaultRoutingSearchParameters()
    solucao_ort = roteamento.SolveWithParameters(parametros_busca)

    # Constrói a rota final
    rota = []
    indice = roteamento.Start(0)
    node = gerenciador.IndexToNode(indice)
    rota.append(node)

    while not roteamento.IsEnd(indice):
        indice = solucao_ort.Value(roteamento.NextVar(indice))
        node = gerenciador.IndexToNode(indice)
        rota.append(node)
    
    return rota, solucao_ort.ObjectiveValue()


# Resolva com nossa matriz de distâncias de antes
tsp_ortools(matriz_distancias)

([0, 3, 2, 1, 0], 70)

# O Problema de Roteamento de Veículos(VRP)

No problema de interesse desse estudo, o "Problema de Roteamento de Veículos"(em inglês Vehicle Routing Problem ou VRP), em vez de caixeiros temos motoristas com seus veículos, e em cada nó eles possuem um objetivo bem específico que consiste em entregar ou buscar alguma coisa. Esta diferença sutil causa uma diferença enorme entre o VRP e o TSP. No TSP, cada caixeiro pode visitar qualquer número de nós e a única restrição é evitar repetição. No VRP, podem existir restrições operacionais que também devem ser respeitadas, como volume máximo do veículo, restrições de tempo e distância, entre outras. 

O VRP pode ser estabelecido como:

"Dado um conjunto de cidades/localizações/nós, as distâncias entre eles e um conjunto de restrições operacionais, qual o melhor conjunto de rotas que visita cada elemento exatamente uma vez, retorna à origem, e respeita as restrições?"

o VRP é, portanto, basicamente um TSP com restrições. Por isso, o procedimento para resolver este tipo de problema utilizando o OR-Tools é essencialmente o mesmo, com a diferença de que é necessário incluir parâmetros com a restrição de capacidade e múltiplos veículos.

In [ ]:
# Nova matriz de distâncias
matriz_distancias = np.array([
    [0, 7, 20, 6],
    [7, 0, 12, 45],
    [20, 12, 0, 55],
    [6, 55, 45, 0],
])

# Demanda de cada nó e capacidade de cada veículo
demanda_no = [0, 4, 3, 4]
capacidade_veiculo = 10

n = matriz_distancias.shape[0]  # número de nós do problema    
no_inicial = 0  
qtde_veiculos = 2
gerenciador = pywrapcp.RoutingIndexManager(n, qtde_veiculos, no_inicial)
roteamento = pywrapcp.RoutingModel(gerenciador)

A restrição de capacidade pode ser incluída de maneira similar à função que retorna a distância entre dois nós, mas ao invés da distância, essa nova função retorna a demanda de cada nó.

In [ ]:
def retorna_demanda(n_index):
    """Retorna a demanda de um nó"""    
    n_no = gerenciador.IndexToNode(n_index)
    return demanda_no[n_no]

demand_callback_index = roteamento.RegisterUnaryTransitCallback(retorna_demanda)
roteamento.AddDimensionWithVehicleCapacity(
    demand_callback_index,
    0,  
    # Cria uma lista que repete a capacidade tantas vezes quanto há veículos
    [capacidade_veiculo] * qtde_veiculos, 
    # Parâmetro de acúmulo
    True,
    'Capacidade'
)

True

Novamente encapsulando os novos parâmetros em uma função, temos o código abaixo que utiliza o OR-Tools para resolver um VRP

In [ ]:
def vrp_ortools(
    matriz_distancias, demanda_no, capacidade_veiculo):
    n = matriz_distancias.shape[0]   
    no_inicial = 0 
    # definimos o número máximo de veículos, um por entrega
    qtde_veiculos = n
    gerenciador = pywrapcp.RoutingIndexManager(n, qtde_veiculos, no_inicial)
    roteamento = pywrapcp.RoutingModel(gerenciador)
    
    def retorna_distancia(i, j):
        ni = gerenciador.IndexToNode(i)
        nj = gerenciador.IndexToNode(j)
        return matriz_distancias[ni, nj]

    transit_callback_index = roteamento.RegisterTransitCallback(retorna_distancia)
    roteamento.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Adiciona a restrição de capacidade
    def retorna_demanda(n_index):
        """Retorna a demanda de um nó"""    
        n_no = gerenciador.IndexToNode(n_index)
        return demanda_no[n_no]

    demand_callback_index = roteamento.RegisterUnaryTransitCallback(
        retorna_demanda                                            )
    roteamento.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0, 
        [capacidade_veiculo] * qtde_veiculos,
        True,
        'Capacidade'
    )

    # Resolve o problema com métodos default
    parametros_busca = pywrapcp.DefaultRoutingSearchParameters()
    solucao_ort = roteamento.SolveWithParameters(parametros_busca)

    # Exceção para quando não há solução
    if not solution:
        return [], -1

    # Constrói as rotas finais
    def cria_rota_veiculo(indice_veiculo):
        rota = []
        indice = roteamento.Start(indice_veiculo)
        no = gerenciador.IndexToNode(indice)
        rota.append(no)

        while not roteamento.IsEnd(indice):
            indice = solucao_ort.Value(roteamento.NextVar(indice))
            no = gerenciador.IndexToNode(indice)
            rota.append(no)
        return rota
    # Lista que armazena as rotas
    rotas = []
    for indice_veiculo in range(qtde_veiculos):
        rota = cria_rota_veiculo(indice_veiculo)
        # Adiciona somente as rotas válidas
        if len(rota) > 2:
            rotas.append(rota)
    
    return rotas, solucao_ort.ObjectiveValue()



In [ ]:
# Resolve o problema com a matriz carregada anteriormente
demanda_no = [0, 4, 3, 4]
capacidade_veiculo = 10
vrp_ortools(matriz_distancias, demanda_no, capacidade_veiculo)

([[0, 2, 1, 0], [0, 3, 0]], 51)

Agora que construímos um algoritmo capaz de resolver o VRP, iremos adaptá-lo para lidar com problemas mais realistas, utilizando instâncias do repositório carregado no início desse estudo. Como uma instância já está carregada, iremos utilizar uma função presente no repositório para calcular uma matriz de distância real utilizando novamente o OSRM, conforme abaixo:

In [ ]:
from loggibud.v1.distances import calculate_distance_matrix_m

# Cria uma lista de pontos contendo a origem e as entregas
pontos = [instancia.origin]
for delivery in instancia.deliveries:
    pontos.append(delivery.point)

# Cria a matriz de distância utilizando OSRM
matriz_distancias = calculate_distance_matrix_m(pontos, config=osrm_config)
matriz_distancias

array([[   0. , 8603.6, 7366.1, ..., 3636. , 2460.1, 2983.7],
       [8267.5,    0. , 2272. , ..., 6726.4, 7048.4, 6074.2],
       [7096. , 2338.1,    0. , ..., 4106.2, 5876.9, 4902.7],
       ...,
       [3775.2, 7094.9, 4604.1, ...,    0. , 1867.5,  994. ],
       [2468.2, 7067. , 5829.5, ..., 1778.6,    0. , 1433.2],
       [3064.2, 6384. , 5146.5, ...,  652.3,  974.2,    0. ]])

E novamente unir o código em uma função capaz de calcular a solução para uma instância do repositório.

In [ ]:
def loggibud_vrp(instancia):
    matriz_distancias = _compute_distance_matrix(instancia)
    demanda_no = _compute_node_demands(instancia)
    capacidade_veiculo = instancia.vehicle_capacity

    # Chama o solver que construímos
    return vrp_ortools(matriz_distancias, demanda_no, capacidade_veiculo)
    
def _compute_distance_matrix(instancia):
    osrm_config= OSRMConfig(host="http://ec2-34-222-175-250.us-west-2.compute.amazonaws.com")
    
    pontos = [instancia.origin]
    for delivery in instancia.deliveries:
        pontos.append(delivery.point)
    
    return calculate_distance_matrix_m(pontos, config=osrm_config)


def _compute_node_demands(instancia):
    """Retorna uma lista com as demandas de cada nó"""
    demanda_no = [0]  # inicializa com a demanda nula da origem
    for delivery in instancia.deliveries:
        demanda_no.append(delivery.size)
    
    return demanda_no

In [ ]:
rotas, distancia = loggibud_vrp(instancia)
rotas, distancia

([[0,
   349,
   792,
   269,
   284,
   304,
   294,
   309,
   310,
   291,
   295,
   279,
   253,
   252,
   301,
   287,
   289,
   255,
   268,
   271,
   302,
   290,
   262,
   259,
   15,
   17,
   12,
   13,
   16,
   14,
   11,
   169,
   173,
   166,
   168,
   862,
   0],
  [0,
   817,
   824,
   812,
   833,
   800,
   815,
   791,
   831,
   814,
   306,
   305,
   299,
   254,
   292,
   248,
   258,
   260,
   280,
   257,
   270,
   296,
   266,
   249,
   267,
   275,
   274,
   277,
   774,
   779,
   829,
   0],
  [0,
   167,
   175,
   174,
   178,
   188,
   171,
   176,
   177,
   184,
   180,
   172,
   183,
   179,
   170,
   182,
   185,
   189,
   165,
   181,
   288,
   283,
   250,
   308,
   307,
   76,
   1004,
   998,
   0],
  [0,
   8,
   110,
   111,
   91,
   127,
   93,
   104,
   297,
   303,
   298,
   264,
   256,
   263,
   251,
   247,
   261,
   285,
   293,
   286,
   281,
   276,
   265,
   272,
   278,
   67,
   125,
   121,
   835,
   282,

Os resultados são de difícil interpretação, uma vez que a saída do algoritmo são apenas os nós de cada rota. Por isso, vamos organizar a saída em um objeto da classe CVRPSolution, definida no repositório.

In [ ]:
from loggibud.v1.types import CVRPSolution, CVRPSolutionVehicle


def _create_cvrp_solution(instancia, rotas):
    veiculos = []
    for rota in rotas:
        veiculo = _create_cvrp_vehicle(instancia, rota)        
        veiculos.append(veiculo)

    # Com os veículos, construímos o objeto `CVRPSolution`
    return CVRPSolution(
        name = instancia.name,
        vehicles = veiculos
    )

def _create_cvrp_vehicle(problem, route):
    """
    Constrói um objeto do tipo `CVRPSolutionVehicle` a partir de uma rota
    """
    entregas = []
    for no in route[1:-1]:
        entregas.append(instancia.deliveries[no - 1])
    
    return CVRPSolutionVehicle(origin=instancia.origin, deliveries=entregas)

Unindo o trecho à nossa função inicial, chegamos em uma função que retorna a saída em uma classe estruturada

In [ ]:
def loggibud_vrp(instancia):
    matriz_distancias = _compute_distance_matrix(instancia)
    demanda_no = _compute_node_demands(instancia)
    capacidade_veiculo = instancia.vehicle_capacity

    # Chama o solver que construímos
    rotas, distancia = vrp_ortools(
        matriz_distancias, demanda_no, capacidade_veiculo)
    
    # Retorna a solução no formato estruturado
    return _create_cvrp_solution(instancia, rotas)
    
def _compute_distance_matrix(instancia):
    osrm_config= OSRMConfig(
        host="http://ec2-34-222-175-250.us-west-2.compute.amazonaws.com")
    
    pontos = [instancia.origin]
    for delivery in instancia.deliveries:
        pontos.append(delivery.point)
    
    return calculate_distance_matrix_m(pontos, config=osrm_config)


def _compute_node_demands(instancia):
    """Retorna uma lista com as demandas de cada nó"""
    demanda_no = [0]  # inicializa com a demanda nula da origem
    for delivery in instancia.deliveries:
        demanda_no.append(delivery.size)
    
    return demanda_no

def _create_cvrp_solution(instancia, rotas):
    veiculos = []
    for rota in rotas:
        veiculo = _create_cvrp_vehicle(instancia, rota)        
        veiculos.append(veiculo)

    # Com os veículos, construímos o objeto `CVRPSolution`
    return CVRPSolution(
        name = instancia.name,
        vehicles = veiculos
    )

def _create_cvrp_vehicle(problem, route):
    """
    Constrói um objeto do tipo `CVRPSolutionVehicle` a partir de uma rota
    """
    entregas = []
    for no in route[1:-1]:
        entregas.append(instancia.deliveries[no - 1])
    
    return CVRPSolutionVehicle(origin=instancia.origin, deliveries=entregas)

In [ ]:
solucao = loggibud_vrp(instancia)
solucao

CVRPSolution(name='cvrp-0-df-0', vehicles=[CVRPSolutionVehicle(origin=Point(lng=-47.802664728268745, lat=-15.657013854445248), deliveries=[Delivery(id='d970d43c10d4e3082c48c32af2ff2607', point=Point(lng=-47.78452088121427, lat=-15.6525522677663), size=2), Delivery(id='49bae82c201fe6c8f14ed50475ab4967', point=Point(lng=-47.6537425277698, lat=-15.608355320776504), size=1), Delivery(id='59f6c2a91484a8ab98359b3093a46146', point=Point(lng=-47.63801873207085, lat=-15.615197100816687), size=4), Delivery(id='e0a4dad3b4b00591c716c02fdad81d95', point=Point(lng=-47.63609990930243, lat=-15.614633607571125), size=1), Delivery(id='bdf0683c93ad06583189d72f2f891c09', point=Point(lng=-47.63533624506079, lat=-15.615404934212668), size=4), Delivery(id='c9156420e097a6e731eaf27e188c2f06', point=Point(lng=-47.63146298173083, lat=-15.609606128988741), size=10), Delivery(id='231aedaef4c1579ad0b4c1bc1c9723f3', point=Point(lng=-47.630801374645515, lat=-15.611282596432059), size=3), Delivery(id='4b690dc5b58a7f10

In [ ]:
# Plota a solução obtida em formato de ruas
plot_cvrp_solution_routes(solucao, config=osrm_config)

## CVRP Dinâmico

O algoritmo que aperfeiçoamos até agora resolve de maneira satisfatória o problema de organizar rotas eficientes que respeitam as restrições operacionais do nosso problema. Porém, ele ainda tem a limitação de necessitar das informações de todas as entregas antes de poder ser executado. Em um cenário real, isso é inviável devido às restrições que apresentamos anteriormente. Portanto, o algoritmo precisa ser capaz de receber um pacote e alocá-lo o mais rápido possível a um veículo, sem conhecer todas as informações.

O propósito desta estudo então é desenvolver um algoritmo que se baseie em informação do passado para elaboração de rotas futuras.

A premissa para utilização este tipo de método é que a distribuição de entregas ao longo dos dias praticamente não varia. Dentro da estrutura de dados do repositório, existem dados de treino com as entregas realizadas diariamente ao longo de 90 dias para cada cidade e em cada região. Dessa forma, podemos utilizar este conjunto de treinamento e desenvolver um modelo capaz de receber novos pacotes e criar rotas semelhantes. Para testar as saídas do modelo, o repositório conta também com outras 30 instâncias na pasta "dev".

## Algoritmo de Clusterização

O algoritmo que iremos utilizar para dividir as regiões de entrega será baseado em uma análise de agrupamentos, que basicamente visam unir conjuntos de pontos em subconjuntos próximos/similares. No caso utilizaremos uma clusterização K-means, onde cada centróide terá como coordenadas a média aritmética das coordenadas de cada ponto em seu cluster.

Iniciaremos importando a biblioteca K-means e carregando uma instância simplificada do repositório. Também iremos reduzir momentaneamente a capacidade dos veículos, já que estamos lidando com menos entregas e a capacidade padrão supera o volume total.

In [ ]:
from sklearn.cluster import KMeans


file_path = "./data/cvrp-instances-1.0/train/df-0/cvrp-0-df-0.json"
instancia = CVRPInstance.from_file(file_path)

# Limita a apenas as primeiras dez entregas
instancia.deliveries = instancia.deliveries[:10]

# Limita a capacidade dos veículos
instancia.vehicle_capacity = 20

O primeiro passo é extrair as coordenadas dos pontos das entregas e convertê-los em um array, que servirá de input para o algoritmo Kmeans. Após isso basta chamar a função e passar os parâmetros do modelo

In [ ]:
pontos = []
for entrega in instancia.deliveries:
    pontos.append((entrega.point.lat, entrega.point.lng))

# Converte a lista em array
pontos = np.array(pontos)

# Salva o modelo em um objeto
modelo = KMeans(n_clusters=3, random_state=0).fit(pontos)

Com o modelo estabelecido, basta utilizar a função predict para avaliar em qual centróide cada nova entrega pertence de acordo com o modelo.

In [ ]:
# Obtem as coordenadas do ponto e converte em um array
ponto_entrega = np.array([(entrega.point.lat, entrega.point.lng)])

# Previsão de qual cluster a entrega pertence
modelo.predict(ponto_entrega)

array([2], dtype=int32)

Com a lógica definida, podemos começar a tratar instâncias completas do repositório. Para buscar as instâncias de treinamento, precisamos de uma função que obtem o caminho da pasta e aplica um loop que itera em cada arquivo.

In [ ]:
from pathlib import Path

def carrega_instancias(train_path_str):
    # Converte o nome da pasta em uma variável `Path`
    train_path = Path(train_path_str)

    instancias = []
    for arquivo_instancia in train_path.iterdir():
        instancia = CVRPInstance.from_file(arquivo_instancia)
        instancias.append(instancia)

    return instancias

instancias = carrega_instancias("./data/cvrp-instances-1.0/train/df-0")

Depois utilizamos o código que obtem os pontos a partir das coordenadas e o encapsulamos em uma função.

In [ ]:
def coordenadas_entrega(instancia):
   pontos = []
   for entrega in instancia.deliveries:
      pontos.append((entrega.point.lat, entrega.point.lng))

   # Converte a lista em array
   pontos = np.array(pontos)
   return pontos

A função final de treino utiliza essa função para construir o modelo com as instâncias reais

In [ ]:
def funcao_treino(instancias, n_clusters):
    # Une os pontos de cada entrega em uma lista
    lista_pontos = []
    for instancia in instancias:
        lista_pontos.extend(coordenadas_entrega(instancia))

    # Constrói o modelo com KMeans
    modelo = KMeans(n_clusters=n_clusters, random_state=0).fit(pontos)
    return modelo

# O modelo recebe como parâmetros as instancias e o número de clusters desejado
modelo = funcao_treino(instancias, 10)
modelo

KMeans(n_clusters=10, random_state=0)

Com a função de treino definida, basta simularmos a chegada de cada pacote com um loop for, e utilizar uma função que recebe o modelo Kmeans e o pacote e tenta alocá-lo a uma sub-região. Vamos carregar uma instância completa e utilizar o código que encapsula todas as funções necessárias. 

In [ ]:
problema = CVRPInstance.from_file("./data/cvrp-instances-1.0/dev/df-0/cvrp-0-df-90.json")

In [ ]:
def vrp_kmeans(problema, modelo):
    # Inicializa o dicionário com sub-rotas
    num_clusters = modelo.n_clusters
    subrotas = {}
    for i in range(num_clusters):
        subrotas[i] = []

    # Inicializa a variável com os veículos completos
    veiculos = []

    # Resolve o problema dinamicamente
    for entrega in problema.deliveries:
        rota(problema, modelo, entrega, subrotas, veiculos)

    # Para cada sub-rota sobrando em `subrotas`, inicia um novo veículo
    for subrota in subrotas.values():
        solucao_veiculo = inicia_veiculo(problema, subrouta)
        veiculos.append(solucao_veiculo)

    # Converte e retorna uma variável do tipo `CVRPSolution`
    return CVRPSolution(name=problem.name, vehicles=vehicles)


def rota(problema, modelo, entrega, subrotas, veiculos):
    ponto_entrega = np.array([(entrega.point.lat, entrega.point.lng)])
    indice_subregiao = modelo.predict(ponto_entrega)[0]

    # Verifica se o veículo tem espaço para o novo pacote
    if (
        calcula_vol_veiculo(subrotas[indice_subregiao]) + entrega.size
        <= problema.vehicle_capacity
       ):
        subrotas[indice_subregiao].append(entrega)
    else:
        # Finaliza a rota atual
        solucao_veiculo = inicia_veiculo(
            problema, subrotas[indice_subregiao]
        )
        veiculos.append(solucao_veiculo)

        # Adiciona um novo veículo à sub-região
        subrotas[indice_subregiao] = [entrega]

def calcula_vol_veiculo(deliveries):
    volume = 0
    for entrega in deliveries:
        volume += entrega.size
    return volume


def inicia_veiculo(problema, entregas_veiculo):
    matriz_distancias = _compute_distance_matrix(problema, entregas_veiculo)
    indices_ordenados, _ = tsp_ortools(matriz_distancias)
    entregas_veiculo_ordenado = []
    # Desconsidera o primeiro e o último índice, o nó de origem
    for indice_ordenado in indices_ordenados[1:-1]:
        entregas_veiculo_ordenado.append(
            entregas_veiculo[indice_ordenado - 1]
        )

    return CVRPSolutionVehicle(
        origin=problema.origin, deliveries=entregas_veiculo_ordenado
    )

def _compute_distance_matrix(problema, entregas_veiculo):
    osrm_config= OSRMConfig(
        host="http://ec2-34-222-175-250.us-west-2.compute.amazonaws.com")

    pontos = [problema.origin]
    for delivery in entregas_veiculo:
        pontos.append(delivery.point)

    return calculate_distance_matrix_m(pontos, config=osrm_config)

# Adicionamos também a função TSP para ordenamento de cada rota
def tsp_ortools(matriz_distancias):
    # Número de nós do problema
    n = matriz_distancias.shape[0] 
    # Número de veículos 
    num_veiculos = 1 
    # Índice do nó que representa o ponto de origem 
    no_inicial = 0 
    gerenciador = pywrapcp.RoutingIndexManager(n, num_veiculos, no_inicial)
    roteamento = pywrapcp.RoutingModel(gerenciador)
    
    def retorna_distancia(i, j):
        ni = gerenciador.IndexToNode(i)
        nj = gerenciador.IndexToNode(j)
        return matriz_distancias[ni, nj]

    transit_callback_index = roteamento.RegisterTransitCallback(retorna_distancia)
    roteamento.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Resolve o problema com métodos padrão
    parametros_busca = pywrapcp.DefaultRoutingSearchParameters()
    solucao_ort = roteamento.SolveWithParameters(parametros_busca)

    # Constrói a rota final
    rota = []
    indice = roteamento.Start(0)
    node = gerenciador.IndexToNode(indice)
    rota.append(node)

    while not roteamento.IsEnd(indice):
        indice = solucao_ort.Value(roteamento.NextVar(indice))
        node = gerenciador.IndexToNode(indice)
        rota.append(node)
    
    return rota, solucao_ort.ObjectiveValue()
